In [1]:
import pandas as pd
pd.options.display.max_rows = 10000
pd.set_option('display.max_colwidth', -1)
from collections import Counter

In [6]:
graph='../csv/2006.csv'
network=graph

In [7]:
df=pd.read_csv(network)

In [8]:
dg=df[['id','modularity_class']].groupby('modularity_class').count()
dg=dg.reset_index(drop=True)
dg.rename(columns={'id':'cluster size'},inplace=True)

In [9]:
df['mod_size']=df.modularity_class.apply(lambda x: dg.iloc[x])

In [10]:
#df.sort_values(['mod_size','degree'],ascending=False)[['label','modularity_class']]

In [11]:
docs=df.label

In [12]:
import re
from nltk import stem
porter = stem.porter.PorterStemmer()

docs_cleaned = [[re.sub('[.!,;?()\W\d]+', '', word) for word in doc.lower().split()] for doc in docs]

In [13]:
stop= open ('stopList.txt').read()
stop_list = set(stop.replace("\n", " ").split())
docs_stemed = [[porter.stem(word) for word in document if word not in stop_list] for document in docs_cleaned]

In [14]:
df['ind']=df.index

In [15]:
df['terms']=df.ind.apply(lambda x: docs_stemed[x])

In [16]:
dg['keyword']=''
dg['key terms']=''

for mod in dg.index:
    mod_terms=[]
    for terms in df[df.modularity_class==mod].terms:
        for term in terms:
            mod_terms.append(term)
            
    words = [word for word in mod_terms if word != '']                
    words_counter = Counter(words)
    top_words=[word for word in words_counter.most_common(5) if word[1]>1]
    dg.set_value(mod,'keyword',top_words)
    
    terms = [words[i]+' '+words[i+1] for i in range(len(words)-1)]
    terms_counter= Counter(terms)
    top_terms=[term for term in terms_counter.most_common(5) if term[1]>1]
    dg.set_value(mod,'key terms',top_terms)

In [17]:
density=df.groupby(['modularity_class'])['triangles'].mean()
dg['cluster density']=(density/max(density)).round(2)
dg=dg.sort_values('cluster density',ascending=False).reset_index(drop=True)

In [23]:
dg.to_csv('../csv/clusters_2006.csv', encoding='ascii')